# Step 1: Prepare a Shopify product inventory snapshot

In this first step we prepare a snapshot of all the products that we wish to train the recommendation model on. The snapshot is a pickled python object containing the following mappings (python dictionaries):
* `category_to_products_dict` - a mapping of a category name to a list of products in that category (i.e. `Category Name` -> `[Product Name]`)
* `title_to_handle_dict` - a mapping of a product name to its Shopify handle (i.e. `Product Name` -> `Product Handle`)
* `handle_to_id_dict` - a mapping of a product's Shopify handle to its Shopify GID (i.e. `Product Handle` -> `Product GID`)

The purpose of each mapping will be explained later at the appropriate moments in the Notebook.

The snapshot can be prepared in different ways depending on individual's needs and requirements. For this purpose, we provide three alternative options for the most common use cases. Regardless of the option you selected, at the end of this step you should have the `inventory_snapshot_path` variable pointing to the location of the pickled product inventory snapshot.

## Option 1: Prepare snapshot of an existing Shopify shop with an empty inventory

In this setup you have an existing Shopify shop without any products and you wish to populate the shop with example products provided by us and then train the recommendation model on those products.

Start by populating the inventory of the Shopify shop with a bunch of products specified in the `./scripts/shopify/inventory.py` file using the `./scripts/shopify/populate_inventory.py` script. The script requires the following environment variables to be set:
* `SHOPIFY_STORE_URL` - the url of the Shopify instance
* `SHOPIFY_ACCESS_TOKEN` - the GraphQL API access token

Note that the GraphQL API access token has to have the following permissions enabled: `write_product_listings`, `read_product_listings`, `write_products`, `read_products`, `write_publications`, `read_publications`.

In [2]:
import os

os.environ["SHOPIFY_STORE_URL"] = "<INSERT_SHOPIFY_STORE_URL>"
os.environ["SHOPIFY_ACCESS_TOKEN"] = "<INSERT_SHOPIFY_ACCESS_TOKEN>"

!python ./scripts/shopify/populate_inventory.py

Total categories to create (after filtering): 10
100%|███████████████████████████████████████████| 10/10 [00:03<00:00,  2.64it/s]
Creating products for category: Clothing (ID: gid://shopify/Collection/640181698891)
100%|███████████████████████████████████████████| 23/23 [00:15<00:00,  1.51it/s]
Creating products for category: Electronics (ID: gid://shopify/Collection/640181731659)
100%|███████████████████████████████████████████| 25/25 [00:16<00:00,  1.48it/s]
Creating products for category: Books (ID: gid://shopify/Collection/640181764427)
100%|███████████████████████████████████████████| 20/20 [00:13<00:00,  1.49it/s]
Creating products for category: Toys (ID: gid://shopify/Collection/640181797195)
100%|███████████████████████████████████████████| 20/20 [00:13<00:00,  1.54it/s]
Creating products for category: Food (ID: gid://shopify/Collection/640181829963)
100%|███████████████████████████████████████████| 27/27 [00:18<00:00,  1.49it/s]
Creating products for category: Automotive (ID: 

Note: if at any point you wish to remove the entire inventory of an existing Shopify shop and start over, use the `./scripts/shopify/cleanup.py` script. The script requires only the `SHOPIFY_STORE_URL` and `SHOPIFY_ACCESS_TOKEN` environment variables to be set.

After the shop is populated with the products, create the inventory snapshot by running the `./scripts/shopify/prepare_inventory_snapshot.py` script providing the path where to save the generated snapshot.

In [3]:
inventory_snapshot_path = "./training/custom_inventory.pkl"

!python ./scripts/shopify/prepare_inventory_snapshot.py $inventory_snapshot_path

Shopify products inventory saved in ./training/custom_inventory.pkl


## Option 2: Prepare snapshot of an existing Shopify shop with your own products

In this setup you have an existing Shopify shop with your own products and you wish to train the recommendation model on those products.

Create the inventory snapshot by running the `./scripts/shopify/prepare_inventory_snapshot.py` script. Since the script requires access to the selected Shopify instance, you need to provide the following environment variables:
* `SHOPIFY_STORE_URL` - the url of the Shopify instance
* `SHOPIFY_ACCESS_TOKEN` - the GraphQL API access token

Note that the GraphQL API access token has to have the following permissions enabled: `read_product_listings`, `read_products`, `read_publications`.

In [4]:
import os

inventory_snapshot_path = "./training/custom_inventory.pkl"

os.environ["SHOPIFY_STORE_URL"] = "<INSERT_SHOPIFY_STORE_URL>"
os.environ["SHOPIFY_ACCESS_TOKEN"] = "<INSERT_SHOPIFY_ACCESS_TOKEN>"

!python ./scripts/shopify/prepare_inventory_snapshot.py $inventory_snapshot_path

Shopify products inventory saved in ./training/custom_inventory.pkl


## Option 3: Use the provided inventory snapshot

In this setup you do not have any existing Shopify shop but you still wish to follow the notebook to experiment with recommendation model training and deployment. In this case simply use the provided inventory snapshot.

In [ ]:
inventory_snapshot_path = "./training/inventory.pkl"

# Step 2: Prepare a product interactions dataset

In this step, we prepare a product interactions dataset that will be used to train the chosen recommendation model. The easiest way to do this is to **generate a synthetic interactions dataset** between the products from the already generated inventory snapshot. However, if you have your own Shopify shop, you might want to **prepare a dataset based on the actual users' interactions** by e.g. dumping the Shopify clickstream (product interaction related) events obtained from the Snowplow tracker. In this case make sure that the dumped interactions dataset file has the same format as the synthetic dataset file described in this step.

Before generating a synthetic interactions dataset, make sure the `inventory_snapshot_path` variable exists and points to the location of the inventory snapshot (as described in the first step). The synthetic dataset is generated with the `./scripts/ml/generate_interactions.py` script. The script uses interaction rules (between products) defined in `./scripts/ml/interaction_rules.py` and the provided products inventory snapshot to generate a reasonable simulation of users' interactions between products.

Note that if you are generating the synthetic interactions dataset for your own products (you picked the 3rd option in the 1st step), then you have to modify the interaction rules specified in `./scripts/ml/interaction_rules.py` to account for product names that exist in your own inventory snapshot.

In [5]:
import os
import tempfile

session_dir = tempfile.mkdtemp()
print("Session directory created in:", session_dir)

interactions_file = os.path.join(session_dir, "interactions_file")
%run ./scripts/ml/generate_interactions.py $inventory_snapshot_path $interactions_file

Session directory created in: /tmp/tmp7whyv9lc
Total interactions generated: 73345
Interactions dataset saved in: /tmp/tmp7whyv9lc/interactions_file


Each row in the generated dataset describes a single interaction between a user and an item at a specific timestamp. Additionally we include the category of an item, which will be used as **a custom feature** to the ML model. This demonstrates the possibilities of including other custom features (such as: the color or the size of an item, the age group of a user etc) in the future.

An important thing to notice is that at this point, the dataset describes the products by their Shopify handles not by their full names. 

In [6]:
import pandas as pd

columns = ["label", "user_id", "item_handle", "timestamp", "item_category"]
interactions_df = pd.read_csv(interactions_file, sep='\t', names=columns)
interactions_df.sample(10)

,label,user_id,item_handle,timestamp,item_category
21993,1,user_45,dashboard-camera,1729615497,Automotive
1551,1,user_4,resistance-bands,1729563047,Sports
7458,1,user_16,cardigan,1730289298,Clothing
66563,1,user_136,compression-socks,1732043198,Health
30524,1,user_61,teddy-bear,1729732999,Toys
1277,1,user_3,slinky,1731314446,Toys
39182,1,user_79,streaming-device,1730220362,Electronics
60635,1,user_123,car-charger,1732006687,Automotive
68216,1,user_139,hand-cream,1731578640,Beauty
43653,1,user_89,bicycle,1731793329,Sports


Note that each interaction row has an additional `label` column that is a binary label indicating whether the given interaction took place or not. This feature is present only because the data preprocessing pipeline, described in the next section, is based mostly on already existing Amazon Reviews Dataset preprocessing scripts from the [recommenders repository](https://github.com/recommenders-team/recommenders/blob/main/recommenders/datasets/amazon_reviews.py), that expect the interaction dataset in this specific format.

# Step 3: Preprocess interactions dataset and split into train/valid/test datasets

The synthetic dataset is similar to the Amazon Reviews Dataset therefore the data preprocessing pipeline consists mostly of already existing transformations implemented by [the Amazon Dataset preprocessing functions](https://github.com/recommenders-team/recommenders/blob/main/recommenders/datasets/amazon_reviews.py) in the [recommenders repository](https://github.com/recommenders-team/recommenders).

In [8]:
from recommenders.datasets.amazon_reviews import data_preprocessing, _create_vocab, _data_generating, _data_processing

For each interaction in the synthetic dataset, we decide whether it will be used for training, validation or testing:

In [9]:
assigned_interactions_file = _data_processing(interactions_file)
print(f"Interactions split file: {assigned_interactions_file}")

Interactions split file: /tmp/tmp7whyv9lc/preprocessed_output


The generated file contains the original interactions dataset with an additional `dataset` column indicating the dataset assignment of each interaction (`train`/`valid`/`test`):

In [10]:
columns = ["dataset", "label", "user_id", "item_handle", "timestamp", "item_category"]
assigned_interactions_df = pd.read_csv(assigned_interactions_file, sep='\t', names=columns, nrows=10000)
assigned_interactions_df.sample(10)

,dataset,label,user_id,item_handle,timestamp,item_category
6953,train,1,user_15,rubik-s-cube,1729546068,Toys
6297,test,1,user_13,kite,1730187789,Toys
4316,train,1,user_10,poetry,1731386933,Books
5592,train,1,user_12,coffee,1730365466,Electronics
3221,train,1,user_7,hand-cream,1731754657,Clothing
683,train,1,user_2,bluetooth-speaker,1731000358,Electronics
5261,train,1,user_11,fantasy,1729783923,Books
5312,train,1,user_11,yo-yo,1730619159,Toys
4403,train,1,user_10,yo-yo,1729795232,Toys
7104,train,1,user_15,rice,1729589668,Food


Next, we use the assignments to split the interactions dataset into training, validation and testing datasets stored in their own separate files:

In [12]:
valid_file = os.path.join(session_dir, 'valid_data')
test_file = os.path.join(session_dir, 'test_data')
train_file = os.path.join(session_dir, 'train_data')
_data_generating(assigned_interactions_file, train_file, valid_file, test_file)

Additionally, the `_data_generating` function expands the history of individual user interactions (with products) into a time based sequence of interactions representing the user's behaviour. As an example, the following interaction history of a particular user:

| item    | timestamp |
| -------- | ------- |
| laptop  | 1    |
| charger | 2     |
| mouse    | 3    |
| keyboard | 4 |

would be transformed into a sequence of interactions:

| item    | timestamp |  history_item  | history_timestamp |
| -------- | ------- | ------- | ------- |
| charger | 2     | laptop | 1 |
| mouse    | 3    | laptop,charger | 1,2 |
| keyboard | 4 |  laptop,charger,mouse | 1,2,3 |

where each line contains an interaction with an item along will all the previous interactions with other items, that directly resulted in a interaction with that specific item.

Below is a peak of the few expended interaction sequences for a particular user:

In [13]:
columns = ["label", "user_id", "item_name", "item_category", "timestamp", "history_items", "history_categories", "history_timestamps"]
train_df = pd.read_csv(train_file, sep='\t', names=columns, nrows=100)
train_df.head()

,label,user_id,item_name,item_category,timestamp,history_items,history_categories,history_timestamps
0,1,user_1,curtains,Home,1730172136,wardrobe,Home,1729233838
1,1,user_1,dishwasher,Home,1730305937,"wardrobe,curtains","Home,Home","1729233838,1730172136"
2,1,user_1,blender,Home,1730514206,"wardrobe,curtains,dishwasher","Home,Home,Home","1729233838,1730172136,1730305937"
3,1,user_1,dining-table,Home,1730980188,"wardrobe,curtains,dishwasher,blender","Home,Home,Home,Home","1729233838,1730172136,1730305937,1730514206"
4,1,user_1,blanket,Home,1729910530,"wardrobe,curtains,dishwasher,blender,dining-table","Home,Home,Home,Home,Home","1729233838,1730172136,1730305937,1730514206,17..."


Next step in dataset preprocessing is generating encodings/vocabularies (in the form of dictionaries) for every categorical features of the dataset. For this (training) dataset the categorical features are: `user_id`, `item_id` and `item_category`. Again, we use the functions provided by the Amazon Dataset processing scripts from the recommenders repository.

In [15]:
user_vocab = os.path.join(session_dir, 'user_vocab.pkl')
item_vocab = os.path.join(session_dir, 'item_vocab.pkl')
cate_vocab = os.path.join(session_dir, 'category_vocab.pkl')
_create_vocab(train_file, user_vocab, item_vocab, cate_vocab)

The generated vocabularies can be easily explored with the following function:

In [16]:
import pickle as pkl

def load_vocab(filename, n=None):
    with open(filename, "rb") as f:
        vocab = list(pkl.load(f).items())
        return vocab if n == None else vocab[:n]

that is used below to extract the first 10 encodings for each categorical feature:

In [17]:
print(f"Users vocabulary (first 10 entries): {load_vocab(user_vocab, n=10)}\n")
print(f"Items vocabulary (first 10 entries): {load_vocab(item_vocab, n=10)}\n")
print(f"Categories vocabulary (first 10 entries): {load_vocab(cate_vocab, n=10)}")

Users vocabulary (first 10 entries): [('user_107', 0), ('user_139', 1), ('user_116', 2), ('user_132', 3), ('user_58', 4), ('user_68', 5), ('user_34', 6), ('user_61', 7), ('user_43', 8), ('user_75', 9)]

Items vocabulary (first 10 entries): [('default_mid', 0), ('protein-powder-1', 1), ('towel-1', 2), ('graphic-novel', 3), ('water-bottle-1', 4), ('face-mask-1', 5), ('car-charger', 6), ('jump-starter', 7), ('vitamin-c', 8), ('milk', 9)]

Categories vocabulary (first 10 entries): [('default_cat', 0), ('Books', 1), ('Clothing', 2), ('Home', 3), ('Beauty', 4), ('Toys', 5), ('Food', 6), ('Automotive', 7), ('Sports', 8), ('Electronics', 9)]


For the purposes of the ML model inference and deployment, we also need to create `item handle -> category name` mappings for all the products in the interactions dataset. The deployment step of the notebook explains the purpose of this dictionary in more detail.

In [18]:
item_category_dict = interactions_df.set_index("item_handle")["item_category"].to_dict()
item_cat_dict_file = os.path.join(session_dir, 'item_cat_dict.pkl')
pkl.dump(item_category_dict, open(item_cat_dict_file, "wb"))

print(f"Item category mappings (first 10 entries): {load_vocab(item_cat_dict_file, n=10)}")

Item category mappings (first 10 entries): [('flashlight', 'Home'), ('jump-starter', 'Automotive'), ('notebook', 'Home'), ('first-aid-kit', 'Sports'), ('toaster', 'Home'), ('car-charger', 'Automotive'), ('fantasy', 'Books'), ('history', 'Books'), ('graphic-novel', 'Books'), ('science-fiction', 'Books')]


The final step of data preparation is enriching validation and testing datasets with negative interaction samples, that will be used for evaluating the model during training (validation dataset) and after training (testing dataset). Specifically for every existing (thus positive) interaction in a given dataset, we generate consecutive `neg_nums_count` (negative) interactions with random items that the user has never interacted with before.

The function for generating negative interactions is presented below. It was extracted from the recommenders repository and then slightly adjusted for the ease of use (the function from the recommenders repository uses constructs such as global variables, which make it harder to use it "outside"):

In [19]:
def _generate_negative_samples_for_file(dataset_lines, f, neg_nums_count, all_items, item_cat_dict):
    for line in dataset_lines:
        # write out the positive sample immediately
        f.write(line)

        # <label=1> <user_id> <item_id> <category_id> <timestamp> ...
        words = line.strip().split("\t")
        positive_item = words[2]
        count = 0
        neg_items = set()
        while count < neg_nums_count:
            # find a random negative item that the user has not interacted with yet
            neg_item = random.choice(all_items)
            if neg_item == positive_item or neg_item in neg_items:
                continue

            count += 1
            neg_items.add(neg_item)

            # append a negative interaction with a selected item
            words[0] = "0"
            words[2] = neg_item
            words[3] = item_cat_dict[neg_item]
            f.write("\t".join(words) + "\n")

Then we use the function to enrich the datasets with the negative interactions in the following way:
* each positive interaction in the validation dataset is followed by `valid_num_ngs` negative interactions
* each positive interaction in the test dataset is followed by `test_num_ngs` negative interactions

In [20]:
valid_num_ngs = 4 # number of negative instances with a positive instance for validation
test_num_ngs = 9 # number of negative instances with a positive instance for testing

items_list = list(interactions_df["item_handle"])
negative_sampled_test_file = os.path.join(session_dir, 'negative_sampled_test_data')
negative_sampled_valid_file = os.path.join(session_dir, 'negative_sampled_valid_data')

with open(valid_file, "r") as f:
    valid_lines = f.readlines()

with open(negative_sampled_valid_file, "w") as f:
    _generate_negative_samples_for_file(valid_lines, f, valid_num_ngs, items_list, item_category_dict)

with open(test_file, "r") as f:
    test_lines = f.readlines()

with open(negative_sampled_test_file, "w") as f:
    _generate_negative_samples_for_file(test_lines, f, test_num_ngs, items_list, item_category_dict)

At this point we have all the necessary datasets prepared:

In [21]:
def count_lines(filename):
    result = !wc -l $filename | cut -d' ' -f1
    return result[0]
    
print(f"Training dataset location: {train_file}")
print(f"Training dataset examples: {count_lines(train_file)}")
print(f"Validation dataset (sampled with {valid_num_ngs} negative interactions) location: {negative_sampled_valid_file}")
print(f"Validation dataset (sampled with {valid_num_ngs} negative interactions) examples: {count_lines(negative_sampled_valid_file)}")
print(f"Testing dataset (sampled with {test_num_ngs} negative interactions) location: {negative_sampled_test_file}")
print(f"Testing dataset (sampled with {test_num_ngs} negative interactions) examples: {count_lines(negative_sampled_test_file)}")

Training dataset location: /tmp/tmp7whyv9lc/train_data
Training dataset examples: 72898
Validation dataset (sampled with 4 negative interactions) location: /tmp/tmp7whyv9lc/negative_sampled_valid_data
Validation dataset (sampled with 4 negative interactions) examples: 745
Testing dataset (sampled with 9 negative interactions) location: /tmp/tmp7whyv9lc/negative_sampled_test_data
Testing dataset (sampled with 9 negative interactions) examples: 1490


We are ready to begin training the recommendation model.

# Step 4: Train the SLi-Rec model

The recommendation model selected for this use case is [SLi-Rec](https://www.microsoft.com/en-us/research/uploads/prod/2019/07/IJCAI19-ready_v1.pdf). The specific implementation is provided by the [recommenders repository](https://github.com/recommenders-team/recommenders/blob/main/recommenders/models/deeprec/models/sequential/sli_rec.py) and the training process mostly follows the steps outlined in the [quickstart notebook](https://github.com/recommenders-team/recommenders/blob/main/examples/00_quick_start/sequential_recsys_amazondataset.ipynb).

In [22]:
import os
import sys
import tensorflow.compat.v1 as tf
tf.get_logger().setLevel('ERROR')

from recommenders.utils.timer import Timer
from recommenders.utils.constants import SEED
from recommenders.models.deeprec.deeprec_utils import prepare_hparams
from recommenders.models.deeprec.models.sequential.sli_rec import SLI_RECModel as SeqModel
from recommenders.models.deeprec.io.sequential_iterator import SequentialIterator

print(f"System version: {sys.version}")
print(f"Tensorflow version: {tf.__version__}")

System version: 3.10.15 (main, Oct  3 2024, 07:27:34) [GCC 11.2.0]
Tensorflow version: 2.12.0


First we prepare the model hyperparameters.

Since our training dataset contains only positive interactions (instances), we need to enable in-training negative sampling by setting the `need_sample` parameter and providing the desired number of negative instances following a positive one with the `train_num_ngs` parameter. This will enable dynamic negative sampling in mini-batches during model training. The training is done in 10 epochs with mini-batches of 400 (negative expanded) interactions, which we found to be a good enough spot for the model to converge and provide decent recommendations for our use case.

The rest of the parameters are either self explanatory or were chosen as defaults from the recommenders repository quickstart notebook. Some of the parameters are provided implicitly via the `yaml_train_config_file` config file, which is mostly a copy of the original file from the recommenders repository as well.

In [23]:
EPOCHS = 10
BATCH_SIZE = 400
yaml_train_config_file = 'training/model_train_config.yaml'
train_num_ngs = 4 # number of negative instances with a positive instance for training

hparams = prepare_hparams(yaml_train_config_file, 
                          embed_l2=0., 
                          layer_l2=0.,
                          learning_rate=0.001,
                          epochs=EPOCHS,
                          batch_size=BATCH_SIZE,
                          show_step=50,
                          MODEL_DIR=os.path.join(session_dir, "training/model"),
                          SUMMARIES_DIR=os.path.join(session_dir, "training/summary/"),
                          user_vocab=user_vocab,
                          item_vocab=item_vocab,
                          cate_vocab=cate_vocab,
                          need_sample=True,
                          train_num_ngs=train_num_ngs,
                         )

Next, we create a model instance:

In [24]:
model = SeqModel(hparams, SequentialIterator, seed=SEED)

Before we start training, we evaluate the model on the test dataset just to get a baseline of how the model is currently performing:

In [25]:
print(model.run_eval(negative_sampled_test_file, num_ngs=test_num_ngs))

{'auc': 0.4905, 'logloss': 0.6931, 'mean_mrr': 0.2597, 'ndcg@2': 0.1264, 'ndcg@4': 0.2138, 'ndcg@6': 0.2815, 'group_auc': 0.4974}


As we can see with AUC=0.5, the untrained model behaves like random guessing whether the user would interact with a given product or not.

Now, lets see if we can improve the model predictions by training it on the training dataset.

In [26]:
with Timer() as train_time:
    model = model.fit(train_file, negative_sampled_valid_file, valid_num_ngs=valid_num_ngs) 

print('Time cost for training is {0:.2f} mins'.format(train_time.interval/60.0))

step 50 , total_loss: 1.6044, data_loss: 1.6044
step 100 , total_loss: 1.5847, data_loss: 1.5847
step 150 , total_loss: 1.5720, data_loss: 1.5720
eval valid at epoch 1: auc:0.6382,logloss:0.6707,mean_mrr:0.5867,ndcg@2:0.5046,ndcg@4:0.6367,ndcg@6:0.6887,group_auc:0.6393
step 50 , total_loss: 1.5576, data_loss: 1.5576
step 100 , total_loss: 1.5604, data_loss: 1.5604
step 150 , total_loss: 1.4980, data_loss: 1.4980
eval valid at epoch 2: auc:0.6432,logloss:0.6434,mean_mrr:0.5824,ndcg@2:0.4997,ndcg@4:0.6573,ndcg@6:0.6859,group_auc:0.6493
step 50 , total_loss: 1.5038, data_loss: 1.5038
step 100 , total_loss: 1.3551, data_loss: 1.3551
step 150 , total_loss: 1.3658, data_loss: 1.3658
eval valid at epoch 3: auc:0.748,logloss:0.583,mean_mrr:0.6937,ndcg@2:0.6551,ndcg@4:0.7554,ndcg@6:0.771,group_auc:0.7768
step 50 , total_loss: 1.3036, data_loss: 1.3036
step 100 , total_loss: 1.2922, data_loss: 1.2922
step 150 , total_loss: 1.2893, data_loss: 1.2893
eval valid at epoch 4: auc:0.7593,logloss:0.590

The training process generated model checkpoints and summaries files inside the `hparams.SUMMARIES_DIR` directory (if it did not, then make sure the `info.write_tfevents` in `yaml_train_config_file` is set to `True`). Those summaries can be later inspected to gain a better understanding of the entire training process via the TensorBoard application:
```shell
$ tensorboard --logdir=<hparams.SUMMARIES_DIR>
```

After the training is done, we are ready to reevaluate our model on the test dataset:

In [27]:
print(model.run_eval(negative_sampled_test_file, num_ngs=test_num_ngs))

{'auc': 0.8634, 'logloss': 0.6669, 'mean_mrr': 0.7408, 'ndcg@2': 0.7264, 'ndcg@4': 0.7941, 'ndcg@6': 0.8041, 'group_auc': 0.9165}


Although further training might slightly improve the model's performance, the current AUC=0.86 is good enough for our use case.

# Step 5: Customize the trained model for deployment

Before we can deploy the trained model, we need to make a few adjustments to make the model work with our use case. The best way of customizing the model logic is by creating a wrapper based on the [mlflow custom python function interface](https://mlflow.org/docs/latest/traditional-ml/creating-custom-pyfunc/index.html). This interface requires the following two functions to be provided:
* `load_context` - invoked on every loading of the model
* `predict` - invoked on every inference request to the model

In the sections below we outline the most important customizations that the provided wrapper implements.

## Model loading

The process of creating an instance of a trained model involves providing the following files:
* model weights obtained from training
* model configuration file - we provide the configuration file for the purpose of model deployment in `./serving/model_serve_config.yaml` file. This is a stripped down version of the configuration file that is used for training
* files generated during dataset preparation:
  * vocabulary files
  * the `item handle -> category name` mappings (`item_cat_dict_file`)
  * the inventory snapshot for the purpose of mapping a product's Shopify handle to its Shopify GID

The trained model instance is recreated inside the `load_context` function.

## Model inference

The model inference is implemented inside the `predict` function. Additional `build_batches_generator` and `encode_single_item` functions are there for the model input preprocessing.

The goal of the model is to provide a list of top N (`TOP_N_HIGHEST_RECOMMENDATIONS`) best recommendations for a given user based on his/her latest sequence of interactions between products. Since the selected model is trained as a binary classifier, it only calculates a probability of a positive interaction with a single given product. Therefore to construct of list of top N best recommendations, it is necassary to infer the model on all the products in the inventory - that is why the model wrapper has to have an access to the products inventory. Additionally to speed up the model inference on the entire products inventory, the products are grouped and inferred in `BATCH_SIZE`ed batches (`build_batches_generator` function). Once the interaction probabilities are inferred on the entire inventory, the list is sorted and the `TOP_N_HIGHEST_RECOMMENDATIONS` products with the highest positive interaction probabilities are selected. As an additional step, the selected products are mapped to their respective Shopify GIDs which are then returned by the model. Note that this Shopify GID mapping is there mostly as a convenient way to avoid doing it inside the Nussknacker scenario itself. 

## Custom item and user features

The model allows the incorporation of custom item and user features, however this is not provided out of the box and requires changes in the model implementation. Features are represented as additional embedding layers of specified sizes that get merged together and placed at the "beginning" of the actual model graph. The process is clearly illustrated in `_build_embedding` and `_lookup_from_embedding` methods of the [SequentialBaseModel](https://github.com/recommenders-team/recommenders/blob/main/recommenders/models/deeprec/models/sequential/sequential_base_model.py) class.

As usual, categorial features require some form of encoding before they are passed to the model. In this example we use vocabularies, generated during model training, based on the known variants (present in the training dataset) of each feature. This is the way we encode the user identifier (`user_vocab` artifact), product Shopify handle (`item_vocab` artifact) and category name (`category_vocab` artifact). The vocabularies are uploaded together with other model artifacts and are used during model inference for encoding provided values of categorical features.

In [28]:
import mlflow

artifacts = {
    "model_data" : os.path.join(session_dir, "training/model/"),
    "model_config": "serving/model_serve_config.yaml",
    "user_vocab" : user_vocab,
    "item_vocab" : item_vocab,
    "category_vocab" : cate_vocab,
    "item_category_dict": item_cat_dict_file,
    "inventory_snapshot": inventory_snapshot_path,
}

class SliRecModelWrapper(mlflow.pyfunc.PythonModel):

    def load_context(self, context):
        from recommenders.models.deeprec.models.sequential.sli_rec import SLI_RECModel as SeqModel
        from recommenders.models.deeprec.io.sequential_iterator import SequentialIterator
        from recommenders.models.deeprec.deeprec_utils import prepare_hparams
        import numpy as np

        hparams = prepare_hparams(
            context.artifacts["model_config"],
            user_vocab=context.artifacts["user_vocab"],
            item_vocab=context.artifacts["item_vocab"],
            cate_vocab=context.artifacts["category_vocab"],
        )
        
        self.model = SeqModel(hparams, SequentialIterator)
        self.model.load_model(context.artifacts["model_data"] + "./artifacts/best_model")
        self.item_cat_dict = self.load_dict(context.artifacts["item_category_dict"])
        self.item_handle_to_id_dict = self.load_dict(context.artifacts["inventory_snapshot"])["handle_to_id_dict"]
        self.TOP_N_HIGHEST_RECOMMENDATIONS = 16
        self.BATCH_SIZE = 16
        
    def predict(self, context, model_input):
        import numpy as np
        import time
        
        user_id = model_input['userId'][0]
        history_item_ids = model_input['items'][0]
        history_timestamps = model_input['timestamps'][0]
        now_timestamp = int(time.time())

        batches = self.build_batches_generator(user_id, now_timestamp, history_item_ids, history_timestamps)

        inferred_items = []
        inferred_preds = []
        for (batch, items) in batches:
            preds = self.model.infer(self.model.sess, batch)
            inferred_preds += (preds[0].flatten().tolist())
            inferred_items += items

        top_n_item_indices = np.argsort(inferred_preds)[::-1][:self.TOP_N_HIGHEST_RECOMMENDATIONS]
        return [{
            "itemIds": list(map(lambda h: self.item_handle_to_id_dict[h], np.array(inferred_items)[top_n_item_indices].tolist())),
            "preds": np.array(inferred_preds)[top_n_item_indices].tolist()
        }]

    def build_batches_generator(self, user_id, now_timestamp, history_item_ids, history_timestamps, min_seq_length=1):
        it = self.model.iterator
        history_item_categories = [self.item_cat_dict[k] for k in history_item_ids]
        
        batched_item_ids = []
        label_list = []
        user_list = []
        item_list = []
        item_cate_list = []
        item_history_batch = []
        item_cate_history_batch = []
        time_list = []
        time_diff_list = []
        time_from_first_action_list = []
        time_to_now_list = []

        cnt = 0
        for item_id, item_category in self.item_cat_dict.items():
            encoded_item = self.encode_single_item(
                user_id,
                item_id,
                item_category,
                now_timestamp,
                history_item_ids,
                history_item_categories,
                history_timestamps
            )

            if len(encoded_item["historyItemIds"]) < min_seq_length:
                continue

            batched_item_ids.append(item_id)
            user_list.append(encoded_item["userId"])
            item_list.append(encoded_item["itemId"])
            item_cate_list.append(encoded_item["itemCategory"])
            item_history_batch.append(encoded_item["historyItemIds"])
            item_cate_history_batch.append(encoded_item["historyCategories"])
            time_list.append(encoded_item["currentTime"])
            time_diff_list.append(encoded_item["timeDiff"])
            time_from_first_action_list.append(encoded_item["timeFromFirstAction"])
            time_to_now_list.append(encoded_item["timeToNow"])

            # label is useless for prediction but required for SliRec conversion utilities
            label_list.append(0)

            cnt += 1
            if cnt == self.BATCH_SIZE:
                res = it._convert_data(
                    label_list,
                    user_list,
                    item_list,
                    item_cate_list,
                    item_history_batch,
                    item_cate_history_batch,
                    time_list,
                    time_diff_list,
                    time_from_first_action_list,
                    time_to_now_list,
                    0,
                )
                batch_feed_dict = it.gen_feed_dict(res)
                yield (batch_feed_dict, batched_item_ids) if batch_feed_dict else None
                
                batched_item_ids = []
                label_list = []
                user_list = []
                item_list = []
                item_cate_list = []
                item_history_batch = []
                item_cate_history_batch = []
                time_list = []
                time_diff_list = []
                time_from_first_action_list = []
                time_to_now_list = []
                cnt = 0
        # process the remaining inputs in the last batch
        if cnt > 0:
            res = it._convert_data(
                label_list,
                user_list,
                item_list,
                item_cate_list,
                item_history_batch,
                item_cate_history_batch,
                time_list,
                time_diff_list,
                time_from_first_action_list,
                time_to_now_list,
                0,
            )
            batch_feed_dict = it.gen_feed_dict(res)
            yield (batch_feed_dict, batched_item_ids) if batch_feed_dict else None

    # extracted and adjusted based on the SequentialIterator from the recommenders module
    # https://github.com/recommenders-team/recommenders/blob/main/recommenders/models/deeprec/io/sequential_iterator.py
    def encode_single_item(self, userId, itemId, itemCategory, nowTimestamp, historyItemIds, historyCategories, historyTimestamps):
        import numpy as np
        
        it = self.model.iterator
        
        user_id = it.userdict[userId] if userId in it.userdict else 0
        item_id = it.itemdict[itemId] if itemId in it.itemdict else 0
        item_cate = it.catedict[itemCategory] if itemCategory in it.catedict else 0
        current_time = float(nowTimestamp)

        item_history_sequence = []
        cate_history_sequence = []
        time_history_sequence = []
    
        for item in historyItemIds:
            item_history_sequence.append(
                it.itemdict[item] if item in it.itemdict else 0
            )
        
        for cate in historyCategories:
            cate_history_sequence.append(
                it.catedict[cate] if cate in it.catedict else 0
            )

        time_history_sequence = [float(i) for i in historyTimestamps]
        time_range = 3600 * 24

        time_diff = []
        for i in range(len(time_history_sequence) - 1):
            diff = (time_history_sequence[i + 1] - time_history_sequence[i]) / time_range
            diff = max(diff, 0.5)
            time_diff.append(diff)
    
        last_diff = (current_time - time_history_sequence[-1]) / time_range
        last_diff = max(last_diff, 0.5)
        time_diff.append(last_diff)
        time_diff = np.log(time_diff)

        time_from_first_action = []
        first_time = time_history_sequence[0]
        time_from_first_action = [
            (t - first_time) / time_range for t in time_history_sequence[1:]
        ]
        time_from_first_action = [max(t, 0.5) for t in time_from_first_action]
        last_diff = (current_time - first_time) / time_range
        last_diff = max(last_diff, 0.5)
        time_from_first_action.append(last_diff)
        time_from_first_action = np.log(time_from_first_action)

        time_to_now = []
        time_to_now = [(current_time - t) / time_range for t in time_history_sequence]
        time_to_now = [max(t, 0.5) for t in time_to_now]
        time_to_now = np.log(time_to_now)

        return {
            "userId": user_id,
            "itemId": item_id,
            "itemCategory": item_cate,
            "historyItemIds": item_history_sequence,
            "historyCategories": cate_history_sequence,
            "currentTime": current_time,
            "timeDiff": time_diff,
            "timeFromFirstAction": time_from_first_action,
            "timeToNow": time_to_now
        }

    def load_dict(self, filename):
        import pickle as pkl
        
        with open(filename, "rb") as f:
            return pkl.load(f)

# Step 6: Log the model to Databricks registry

The last step is to deploy the model wrapper to the Databricks registry. When executing the notebook inside the Databricks environment, the deployment process is trivial and comes down to invoking the `mlflow.pyfunc.log_model` function that accepts:
* the name of the model - this is the name that will be used later to refer to the model via the MLFlow enricher
* model artifacts - those are all the files needed to recreate an instance of the model (explained in more detail in step 5)
* model signature - the example based signature for the model
* conda environment - hints about the required dependencies needed by the model

In [ ]:
import mlflow
from mlflow.models import infer_signature

mlflow.set_registry_uri("databricks")

signature = infer_signature(model_input = {
    "userId": "<not-important-user-id>",
    "items": ["thermometer", "comic-book", "brush"],
    "timestamps": ["1730807061", "1730808119", "1730809203"]
}, model_output = {
    "itemIds": ["gid://Product/14809004540249", "gid://Product/14809003753817", "gid://Product/14809013879129"],
    "preds": [0.9762271046638489, 0.8966923356056213, 0.6756445169448853]
})

default_conda_env = mlflow.pyfunc.get_default_conda_env()
default_conda_env['dependencies'].append('tensorflow=2.12.1')
default_conda_env['dependencies'].append('recommenders=1.2.0')

with mlflow.start_run():
    mlflow.pyfunc.log_model(
        artifact_path="slirec_shopify_model",
        python_model=SliRecModelWrapper(),
        conda_env=default_conda_env,
        artifacts=artifacts,
        registered_model_name="slirec_shopify_model",
        signature=signature
    )

/home/mst/anaconda3/envs/test_rec_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Uploading artifacts: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:03<00:00,  3.58it/s]
Registered model 'slirec_shopify_model' already exists. Creating a new version of this model...
2024/11/15 17:25:31 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: slirec_shopify_model, version 2
Created version '2' of model 'slirec_shopify_model'.


The last line of the deployment log shows the name of the deployed model along with the specific version that was assigned to the model. Knowing this information allows us to select the appropriate model in the MLFlow enricher. 